# Import Dependencies and Read CSV

In [163]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import ExtraTreesRegressor
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt


#  Import and read the csv.
import pandas as pd 
import os

### Use on Google Colab
import io
from google.colab import files
uploaded = files.upload()
co_covid_df= pd.read_csv(io.BytesIO(uploaded['CleanCovidData.csv']))
co_covid_df.head()



Saving CleanCovidData.csv to CleanCovidData (6).csv


,County,oneweek_cumulative_incidence,twoweek_cumulative_incidence,oneweek_average_positivity,twoweek_average_positivity,county_size,fips,administered_dose1_recip,administered_dose1_pop_pct,series_complete_yes,series_complete_pop_pct,svi_ctgy,series_complete_pop_pct_svi,metro_status,series_complete_pop_pct_ur_equity,census2019
0,Adams County,349.0,650.5,0.106,0.101,">30,000",8001,333369,64.4,309975,59.9,C,12,Metro,4,517421
1,Alamosa County,655.3,1224.8,0.077,0.088,"30,000 or less",8003,8948,55.1,8162,50.3,D,16,Non-metro,8,16233
2,Arapahoe County,313.1,581.0,0.093,0.083,">30,000",8005,420590,64.1,394045,60.0,B,8,Metro,4,656590
3,Archuleta County,568.5,897.7,0.172,0.148,"30,000 or less",8007,8183,58.3,7446,53.1,B,8,Non-metro,8,14029
4,Baca County,1265.8,1841.2,0.070,0.065,"30,000 or less",8009,1360,38.0,1322,36.9,C,10,Non-metro,6,3581


# Clean the Data

In [164]:
clean_covid_load = os.path.join("covid19_county_metrics_mod.csv")

In [165]:
#co_covid_df = pd.read_csv(clean_covid_load)
#co_covid_df

In [166]:
co_covid_df = co_covid_df.rename(columns = {'county':'changeme'})
co_covid_df

,County,oneweek_cumulative_incidence,twoweek_cumulative_incidence,oneweek_average_positivity,twoweek_average_positivity,county_size,fips,administered_dose1_recip,administered_dose1_pop_pct,series_complete_yes,series_complete_pop_pct,svi_ctgy,series_complete_pop_pct_svi,metro_status,series_complete_pop_pct_ur_equity,census2019
0,Adams County,349.0,650.5,0.106,0.101,">30,000",8001,333369,64.4,309975,59.9,C,12,Metro,4,517421
1,Alamosa County,655.3,1224.8,0.077,0.088,"30,000 or less",8003,8948,55.1,8162,50.3,D,16,Non-metro,8,16233
2,Arapahoe County,313.1,581.0,0.093,0.083,">30,000",8005,420590,64.1,394045,60.0,B,8,Metro,4,656590
3,Archuleta County,568.5,897.7,0.172,0.148,"30,000 or less",8007,8183,58.3,7446,53.1,B,8,Non-metro,8,14029
4,Baca County,1265.8,1841.2,0.070,0.065,"30,000 or less",8009,1360,38.0,1322,36.9,C,10,Non-metro,6,3581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,Summit County,364.3,677.0,0.117,0.117,">30,000",8117,26541,85.6,23785,76.7,A,4,Non-metro,8,31011
60,Teller County,270.4,670.0,0.109,0.117,"30,000 or less",8119,13250,52.2,12635,49.8,A,3,Metro,3,25388
61,Washington County,393.5,931.9,0.119,0.106,"30,000 or less",8121,1500,30.6,1353,27.6,B,5,Non-metro,5,4908
62,Weld County,444.2,844.8,0.125,0.119,">30,000",8123,187498,57.8,173708,53.5,B,8,Metro,4,324492


In [167]:
#co_covid_df['County'] = co_covid_df['changeme'].astype(str) + ' County'
#co_covid_df
#co_covid_df.drop(['changeme'], axis=1, inplace=True)


In [168]:
co_covid_df = co_covid_df.rename(columns = {"One-Week Cumulative Incidence":"OneWeek_Cumulative_Incidence", "Two-Week Cumulative Incidence":"TwoWeek_Cumulative_Incidence", "One-Week Average Positivity":"OneWeek_Average Positivity", "Two-Week Average Positivity":"TwoWeek_Average_Positivity", "County size":"County_size"})
co_covid_df

,County,oneweek_cumulative_incidence,twoweek_cumulative_incidence,oneweek_average_positivity,twoweek_average_positivity,county_size,fips,administered_dose1_recip,administered_dose1_pop_pct,series_complete_yes,series_complete_pop_pct,svi_ctgy,series_complete_pop_pct_svi,metro_status,series_complete_pop_pct_ur_equity,census2019
0,Adams County,349.0,650.5,0.106,0.101,">30,000",8001,333369,64.4,309975,59.9,C,12,Metro,4,517421
1,Alamosa County,655.3,1224.8,0.077,0.088,"30,000 or less",8003,8948,55.1,8162,50.3,D,16,Non-metro,8,16233
2,Arapahoe County,313.1,581.0,0.093,0.083,">30,000",8005,420590,64.1,394045,60.0,B,8,Metro,4,656590
3,Archuleta County,568.5,897.7,0.172,0.148,"30,000 or less",8007,8183,58.3,7446,53.1,B,8,Non-metro,8,14029
4,Baca County,1265.8,1841.2,0.070,0.065,"30,000 or less",8009,1360,38.0,1322,36.9,C,10,Non-metro,6,3581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,Summit County,364.3,677.0,0.117,0.117,">30,000",8117,26541,85.6,23785,76.7,A,4,Non-metro,8,31011
60,Teller County,270.4,670.0,0.109,0.117,"30,000 or less",8119,13250,52.2,12635,49.8,A,3,Metro,3,25388
61,Washington County,393.5,931.9,0.119,0.106,"30,000 or less",8121,1500,30.6,1353,27.6,B,5,Non-metro,5,4908
62,Weld County,444.2,844.8,0.125,0.119,">30,000",8123,187498,57.8,173708,53.5,B,8,Metro,4,324492


In [169]:
co_covid_df.dtypes

County                                object
oneweek_cumulative_incidence         float64
twoweek_cumulative_incidence         float64
oneweek_average_positivity           float64
twoweek_average_positivity           float64
county_size                           object
fips                                   int64
administered_dose1_recip               int64
administered_dose1_pop_pct           float64
series_complete_yes                    int64
series_complete_pop_pct              float64
svi_ctgy                              object
series_complete_pop_pct_svi            int64
metro_status                          object
series_complete_pop_pct_ur_equity      int64
census2019                             int64
dtype: object

In [170]:
# Find metro status values
co_covid_df['metro_status'].unique

<bound method Series.unique of 0         Metro
1     Non-metro
2         Metro
3     Non-metro
4     Non-metro
        ...    
59    Non-metro
60        Metro
61    Non-metro
62        Metro
63    Non-metro
Name: metro_status, Length: 64, dtype: object>

In [171]:
# Encode metro status string values into numeric values
metro_dict = {'Non-metro': 0, 'Metro': 1}
co_covid_df = co_covid_df.replace({'metro_status': metro_dict})

In [172]:
# Find values of county size
co_covid_df['county_size'].unique

<bound method Series.unique of 0            >30,000
1     30,000 or less
2            >30,000
3     30,000 or less
4     30,000 or less
           ...      
59           >30,000
60    30,000 or less
61    30,000 or less
62           >30,000
63    30,000 or less
Name: county_size, Length: 64, dtype: object>

In [173]:
# Encode county size string values into numeric values
county_dict = {'30,000 or less': 0, '>30,000': 1}
co_covid_df = co_covid_df.replace({'county_size': county_dict})


In [174]:
# Find values of svi category
co_covid_df['svi_ctgy'].unique

<bound method Series.unique of 0     C
1     D
2     B
3     B
4     C
     ..
59    A
60    A
61    B
62    B
63    C
Name: svi_ctgy, Length: 64, dtype: object>

In [175]:
# Encode svi category string values into numeric values
svi_dict = {'A': 0, 'B': 1, 'C': 3, 'D':4}
co_covid_df = co_covid_df.replace({'svi_ctgy': svi_dict})

In [176]:
# Relevant Features 
co_covid_df = co_covid_df[['County', 'twoweek_average_positivity', 'county_size', 'series_complete_yes', 'series_complete_pop_pct', 'metro_status', 'census2019']]
co_covid_df

,County,twoweek_average_positivity,county_size,series_complete_yes,series_complete_pop_pct,metro_status,census2019
0,Adams County,0.101,1,309975,59.9,1,517421
1,Alamosa County,0.088,0,8162,50.3,0,16233
2,Arapahoe County,0.083,1,394045,60.0,1,656590
3,Archuleta County,0.148,0,7446,53.1,0,14029
4,Baca County,0.065,0,1322,36.9,0,3581
...,...,...,...,...,...,...,...
59,Summit County,0.117,1,23785,76.7,0,31011
60,Teller County,0.117,0,12635,49.8,1,25388
61,Washington County,0.106,0,1353,27.6,0,4908
62,Weld County,0.119,1,173708,53.5,1,324492


## Split the Data into Training and Testing 

In [177]:
# Generate the categorical variable list
feat_cat = ['County','twoweek_average_positivity', 'series_complete_yes', 'census2019']

In [178]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

In [179]:
# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(co_covid_df[feat_cat]))

In [180]:
encode_df.columns = enc.get_feature_names(feat_cat)
encode_df.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,County_Adams County,County_Alamosa County,County_Arapahoe County,County_Archuleta County,County_Baca County,County_Bent County,County_Boulder County,County_Broomfield County,County_Chaffee County,County_Cheyenne County,County_Clear Creek County,County_Conejos County,County_Costilla County,County_Crowley County,County_Custer County,County_Delta County,County_Denver County,County_Dolores County,County_Douglas County,County_Eagle County,County_El Paso County,County_Elbert County,County_Fremont County,County_Garfield County,County_Gilpin County,County_Grand County,County_Gunnison County,County_Hinsdale County,County_Huerfano County,County_Jackson County,County_Jefferson County,County_Kiowa County,County_Kit Carson County,County_La Plata County,County_Lake County,County_Larimer County,County_Las Animas County,County_Lincoln County,County_Logan County,County_Mesa County,...,census2019_8205,census2019_9700,census2019_10019,census2019_11267,census2019_12172,census2019_13283,census2019_14029,census2019_14506,census2019_15734,census2019_16233,census2019_17462,census2019_17767,census2019_18278,census2019_18845,census2019_20356,census2019_22409,census2019_25388,census2019_25638,census2019_26183,census2019_26729,census2019_29068,census2019_31011,census2019_31162,census2019_42758,census2019_47839,census2019_55127,census2019_56221,census2019_60061,census2019_70465,census2019_154210,census2019_168424,census2019_324492,census2019_326196,census2019_351154,census2019_356899,census2019_517421,census2019_582881,census2019_656590,census2019_720403,census2019_727211
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [189]:
# Merge one-hot encoded features and drop the originals
co_covid_df2 = co_covid_df.merge(encode_df, left_index=True, right_index=True).drop(columns=feat_cat, axis=1)
co_covid_df2

,county_size,series_complete_pop_pct,metro_status,County_Adams County,County_Alamosa County,County_Arapahoe County,County_Archuleta County,County_Baca County,County_Bent County,County_Boulder County,County_Broomfield County,County_Chaffee County,County_Cheyenne County,County_Clear Creek County,County_Conejos County,County_Costilla County,County_Crowley County,County_Custer County,County_Delta County,County_Denver County,County_Dolores County,County_Douglas County,County_Eagle County,County_El Paso County,County_Elbert County,County_Fremont County,County_Garfield County,County_Gilpin County,County_Grand County,County_Gunnison County,County_Hinsdale County,County_Huerfano County,County_Jackson County,County_Jefferson County,County_Kiowa County,County_Kit Carson County,County_La Plata County,County_Lake County,County_Larimer County,County_Las Animas County,...,census2019_8205,census2019_9700,census2019_10019,census2019_11267,census2019_12172,census2019_13283,census2019_14029,census2019_14506,census2019_15734,census2019_16233,census2019_17462,census2019_17767,census2019_18278,census2019_18845,census2019_20356,census2019_22409,census2019_25388,census2019_25638,census2019_26183,census2019_26729,census2019_29068,census2019_31011,census2019_31162,census2019_42758,census2019_47839,census2019_55127,census2019_56221,census2019_60061,census2019_70465,census2019_154210,census2019_168424,census2019_324492,census2019_326196,census2019_351154,census2019_356899,census2019_517421,census2019_582881,census2019_656590,census2019_720403,census2019_727211
0,1,59.9,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0,50.3,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,60.0,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0,53.1,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,36.9,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,1,76.7,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60,0,49.8,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.

In [188]:
#co_covid_df2.corr()

In [190]:
# Split our preprocessed data into our features and target arrays
y = co_covid_df2["series_complete_pop_pct"]
X = co_covid_df2.drop("series_complete_pop_pct", axis=1)

In [191]:
# Split the preprocessed data into a training and testing dataset
# The training and testing data is split into 75% and 25%, respectively.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

ValueError: ignored

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

In [ ]:
# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

In [ ]:
# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Undersampling, We are using undersampling to normalize our data.

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
ros = RandomUnderSampler(random_state=1)
X_resampled2, y_resampled2 = ros.fit_resample(X_train_scaled, y_train)
Counter(y_resampled)

In [ ]:
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled2, y_resampled2)

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test_scaled)
confusion_matrix(y_test, y_pred)

In [ ]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

In [ ]:
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

# Decision Tree, We have decided to use a decision tree model for our Machine Learning Model because we are predicting a binary outcome.

In [ ]:
# Import Dependencies
from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
# Creating the decision tree classifier instance.
model = tree.DecisionTreeClassifier()
# Fitting the model.
model = model.fit(X_train_scaled, y_train)

In [ ]:
# Making predictions using the testing data.
predictions = model.predict(X_test_scaled)

In [ ]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)

In [ ]:
# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

In [ ]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions

In [ ]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

# Logistic Regression, We also wanted to look at how the model performed for a Logistic Regression model because it also predicts binary values.

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs',
   max_iter=200,
   random_state=1)

In [ ]:
classifier.fit(X_resampled2, y_resampled2)

In [ ]:
y_pred = classifier.predict(X_test_scaled)

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

In [ ]:
#co_covid_df.to_csv('CleanCountyCoviddata.csv', columns=["County", "date", "OneWeek_Cumulative_Incidence", "TwoWeek_Cumulative_Incidence", "OneWeek_Average Positivity", "TwoWeek_Average_Positivity", "County_size"])